# Custom Chatbot Project

TODO: In this cell, write an explanation of which dataset you have chosen and why it is appropriate for this task

## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [1]:
import requests
import pandas as pd

# This function is to fetch Wikipedia data extract for"2022" Wikipedia page
def fetch_wikipedia_extract(title):
    endpoint = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "prop": "extracts",
        "format": "json",
        "titles": title,
        "explaintext": True
    }

    response = requests.get(endpoint, params=params)
    
    if response.status_code == 200:
        data = response.json()
        page = next(iter(data['query']['pages'].values()))
        return page.get('extract', '')
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
        return ""

content = fetch_wikipedia_extract("2022")

paragraphs = content.split("\n")

while len(paragraphs) < 20:
    paragraphs.extend(paragraphs)

paragraphs = paragraphs[:20]

df = pd.DataFrame(paragraphs, columns=["text"])

print(df)

                                                 text
0   2022 (MMXXII) was a common year starting on Sa...
1   The year saw the removal of nearly all COVID-1...
2   2022 was also dominated by wars and armed conf...
3                                                    
4                                                    
5                                        == Events ==
6                                                    
7                                                    
8                                     === January ===
9    January 1 – The Regional Comprehensive Econom...
10  January 2 – Abdalla Hamdok resigns as Prime Mi...
11  January 4 – The five permanent members of the ...
12  January 5 – A nationwide state of emergency is...
13  January 6 – The CSTO deploys a "peacekeeping" ...
14  January 7 – COVID-19 pandemic: The number of C...
15  January 9 – February 6 – The 2021 Africa Cup o...
16  January 10 – The first successful heart transp...
17  January 15 – A large eru

In [2]:
#This step is remove blanks
df = df[df["text"].str.len()>0]
df

,text
0,2022 (MMXXII) was a common year starting on Sa...
1,The year saw the removal of nearly all COVID-1...
2,2022 was also dominated by wars and armed conf...
5,== Events ==
8,=== January ===
9,January 1 – The Regional Comprehensive Econom...
10,January 2 – Abdalla Hamdok resigns as Prime Mi...
11,January 4 – The five permanent members of the ...
12,January 5 – A nationwide state of emergency is...
13,"January 6 – The CSTO deploys a ""peacekeeping"" ..."


In [3]:
#This step is to remove text starting from "=="
df = df[~df["text"].str.startswith("==")]

In [4]:
df.tail(20)

,text
0,2022 (MMXXII) was a common year starting on Sa...
1,The year saw the removal of nearly all COVID-1...
2,2022 was also dominated by wars and armed conf...
9,January 1 – The Regional Comprehensive Econom...
10,January 2 – Abdalla Hamdok resigns as Prime Mi...
11,January 4 – The five permanent members of the ...
12,January 5 – A nationwide state of emergency is...
13,"January 6 – The CSTO deploys a ""peacekeeping"" ..."
14,January 7 – COVID-19 pandemic: The number of C...
15,January 9 – February 6 – The 2021 Africa Cup o...


In [5]:
# This step is to adjust the text for example- If the row's text is not a date then we'll add the prefix "-"
from dateutil.parser import parse

prefix = ""
for (i, row) in df.iterrows():

    if " – " not in row["text"]:
        try:
            parse(row["text"])
            prefix = row["text"]
        except:
            row["text"] = prefix + " – " + row["text"]
df = df[df["text"].str.contains(" – ")].reset_index(drop=True)

In [6]:
df

,text
0,– 2022 (MMXXII) was a common year starting on...
1,– The year saw the removal of nearly all COVI...
2,– 2022 was also dominated by wars and armed c...
3,January 1 – The Regional Comprehensive Econom...
4,January 2 – Abdalla Hamdok resigns as Prime Mi...
5,January 4 – The five permanent members of the ...
6,January 5 – A nationwide state of emergency is...
7,"January 6 – The CSTO deploys a ""peacekeeping"" ..."
8,January 7 – COVID-19 pandemic: The number of C...
9,January 9 – February 6 – The 2021 Africa Cup o...


## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [7]:
import openai

# Set your OpenAI API key
api_key = 'YOUR API KEY'
openai.api_key = api_key

In [8]:
# This step is to create function for custom query
def custom_query(prompt, model="gpt-3.5-turbo"):  
    response = openai.Completion.create(
        model=model,
        prompt=prompt,
        max_tokens=150
    )
    return response.choices[0].text.strip()


In [9]:
# This step is to create sample prompt using the dataset
sample_prompt = f"Here is the paragraphs from the Wikipedia article on the year 2022:\n\n"
for i, row in df.iterrows():
    sample_prompt += f"Paragraph {i+1}:\n{row['text'][:200]}...\n\n" 

sample_prompt += "Answer the following questions based on the above paragraphs."

custom_response = custom_query(sample_prompt)
print(custom_response)


In [ ]:
questions = [
    "What has happened in sports in 2022?"
]


def demonstrate_performance(questions, custom_prompt, model="gpt-3.5-turbo"):
    results = {}
    for question in questions:
        basic_response = custom_query(question, model)
        
        custom_response = custom_query(custom_prompt + "\n\n" + question, model)
        
        results[question] = {
            "basic": basic_response,
            "custom": custom_response
        }
    return results

performance_results = demonstrate_performance(questions, sample_prompt)

for question, result in performance_results.items():
    print(f"Question: {question}")
    print(f"Basic Model Answer: {result['basic']}")
    print(f"Custom Model Answer: {result['custom']}\n")


## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

### Question 1

In [ ]:
questions = [
    "What was the impact of COVID-19 in 2022?"
]


def demonstrate_performance(questions, custom_prompt, model="gpt-3.5-turbo"):
    results = {}
    for question in questions:
        basic_response = custom_query(question, model)
        
        custom_response = custom_query(custom_prompt + "\n\n" + question, model)
        
        results[question] = {
            "basic": basic_response,
            "custom": custom_response
        }
    return results

performance_results = demonstrate_performance(questions, sample_prompt)

for question, result in performance_results.items():
    print(f"Question: {question}")
    print(f"Basic Model Answer: {result['basic']}")
    print(f"Custom Model Answer: {result['custom']}\n")

In [ ]:
questions = [
    "Who are the five permanent members of the UN Security Council?"
]


def demonstrate_performance(questions, custom_prompt, model="gpt-3.5-turbo"):
    results = {}
    for question in questions:
        basic_response = custom_query(question, model)
        
        custom_response = custom_query(custom_prompt + "\n\n" + question, model)
        
        results[question] = {
            "basic": basic_response,
            "custom": custom_response
        }
    return results

performance_results = demonstrate_performance(questions, sample_prompt)

for question, result in performance_results.items():
    print(f"Question: {question}")
    print(f"Basic Model Answer: {result['basic']}")
    print(f"Custom Model Answer: {result['custom']}\n")

### Question 2

In [ ]:
questions = [
    "Abdalla Hamdok has resigned from which position in 2022?"
]


def demonstrate_performance(questions, custom_prompt, model="gpt-3.5-turbo"):
    results = {}
    for question in questions:
        basic_response = custom_query(question, model)
        
        custom_response = custom_query(custom_prompt + "\n\n" + question, model)
        
        results[question] = {
            "basic": basic_response,
            "custom": custom_response
        }
    return results

performance_results = demonstrate_performance(questions, sample_prompt)

for question, result in performance_results.items():
    print(f"Question: {question}")
    print(f"Basic Model Answer: {result['basic']}")
    print(f"Custom Model Answer: {result['custom']}\n")

In [ ]:
questions = [
    "where was the first successful heart transplant from a pig to a human patient occured?"
]


def demonstrate_performance(questions, custom_prompt, model="gpt-3.5-turbo"):
    results = {}
    for question in questions:
        basic_response = custom_query(question, model)
        
        custom_response = custom_query(custom_prompt + "\n\n" + question, model)
        
        results[question] = {
            "basic": basic_response,
            "custom": custom_response
        }
    return results

performance_results = demonstrate_performance(questions, sample_prompt)

for question, result in performance_results.items():
    print(f"Question: {question}")
    print(f"Basic Model Answer: {result['basic']}")
    print(f"Custom Model Answer: {result['custom']}\n")